In [202]:
%load_ext autoreload
%autoreload 2

from shrimpgrad.tensor import Tensor
import torch

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [208]:
x = Tensor.rand(2,2)

(2, 2)


In [199]:
x

tensor([[0.4494672063906652,
  0.5700686239282257,
  0.7478985585775664,
  0.7171695791541391,
  0.8358677962187998,
  0.03055485090914567,
  0.45419554919237637,
  0.3831332166658131,
  0.6210702131709483,
  0.3985247823688609],
 [0.05898549372278872,
  0.6167354159459347,
  0.9211972330195551,
  0.12972497974231612,
  0.025657928319137713,
  0.3514266921918734,
  0.6288923717213797,
  0.08756940582835393,
  0.08183712016023881,
  0.0026299968983352073],
 [0.5715157047736316,
  0.9279704034319675,
  0.09140381712822254,
  0.05388817407023805,
  0.2719516444846851,
  0.9824578157708189,
  0.8089236604137569,
  0.1870107762582509,
  0.11639840274457292,
  0.6514739132148408],
 [0.793000660705823,
  0.1992183925922033,
  0.8857728844876441,
  0.07792257612869802,
  0.4877276785620116,
  0.40552839574275845,
  0.5514190034979165,
  0.04879232613954698,
  0.659086838159407,
  0.9461528298561118],
 [0.040625927223475355,
  0.6149632344190623,
  0.7861667796419879,
  0.6755433664778886,
  0.